In this homework, you can pick one of the two sections (Automated Fact-checking or Relatio) to get the full completion point.

# Automated Fact-checking
In the notebook, we see one off-the-shelf fact-check model based on RoBERTa.
However, that setting is more like textual entailment. Real-world Fact-checking pipeline requires an extra module: evidence retrieval. In this homework, we will add an evidence retrieval model based on <b>sentence-bert</b> to the RoBERTa Fact-checker.

Note: SBERT was introduced in Notebook 6 (06_transformers.ipynb).

In [1]:
!wget https://scifact.s3-us-west-2.amazonaws.com/release/latest/data.tar.gz
!tar -xzf data.tar.gz

--2023-05-25 06:40:06--  https://scifact.s3-us-west-2.amazonaws.com/release/latest/data.tar.gz
Resolving scifact.s3-us-west-2.amazonaws.com (scifact.s3-us-west-2.amazonaws.com)... 3.5.84.112, 3.5.77.189, 3.5.83.150, ...
Connecting to scifact.s3-us-west-2.amazonaws.com (scifact.s3-us-west-2.amazonaws.com)|3.5.84.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3115079 (3.0M) [application/x-tar]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>]   2.97M  3.59MB/s    in 0.8s    

2023-05-25 06:40:07 (3.59 MB/s) - ‘data.tar.gz’ saved [3115079/3115079]



In [2]:
import json

claim_file = 'data/claims_dev.jsonl'
corpus_file = 'data/corpus.jsonl'

corpus = {}
with open(corpus_file) as f:
    for line in f:
        abstract = json.loads(line)
        corpus[str(abstract["doc_id"])] = abstract
        
claims = []
with open(claim_file) as f:
    for line in f:
        claim = json.loads(line)
        claims.append(claim)

print(claims[1])
print(corpus['14717500'])

print("Number of Corpus: ", len(corpus))

{'id': 3, 'claim': '1,000 genomes project enables mapping of genetic sequence variation consisting of rare variants with larger penetrance effects than common variants.', 'evidence': {'14717500': [{'sentences': [2, 5], 'label': 'SUPPORT'}, {'sentences': [7], 'label': 'SUPPORT'}]}, 'cited_doc_ids': [14717500]}
{'doc_id': 14717500, 'title': 'Rare Variants Create Synthetic Genome-Wide Associations', 'abstract': ['Genome-wide association studies (GWAS) have now identified at least 2,000 common variants that appear associated with common diseases or related traits (http://www.genome.gov/gwastudies), hundreds of which have been convincingly replicated.', 'It is generally thought that the associated markers reflect the effect of a nearby common (minor allele frequency >0.05) causal site, which is associated with the marker, leading to extensive resequencing efforts to find causal sites.', 'We propose as an alternative explanation that variants much less common than the associated one may crea

In [3]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm


def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

# utility function for cosine similarity calculation
def cosine_similarity_matrix(vector, matrix):
    return np.apply_along_axis(cosine_similarity, 1, matrix, vector)

# preprocessing function for SciFact corpus
def preprocess_sentence(text):
    text = text.replace('/', ' / ')
    text = text.replace('.-', ' .- ')
    text = text.replace('.', ' . ')
    text = text.replace('\'', ' \' ')
    text = text.lower()

    return text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=904ae8b2ad5baf3505d83cb71dfc45a09769b4687023fee9449055ebb0e9cd26
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


### TODO 1: find the top-10 evidences for the following claim USING SBERT.

In [4]:
claim_1 = claims[1]['claim']

corpus_ids, corpus_texts = [], []
for k, v in corpus.items():
  original_sentences = [v['title']] + v['abstract']
  processed_paragraph = " ".join([preprocess_sentence(sentence) for sentence in original_sentences])
  corpus_ids.append(k)
  corpus_texts.append(processed_paragraph)

# TODO: find the top-10 evidences for claim_1 using SBERT
# Hint 1: use Colab's GPU (or your local GPUs) to accelerate SBERT Encoding
# Hint 2: use parallel encoding (i.e. batch encoding) from SBERT to accelerate encoding.
# Hint 3: SciFact is a scientific domain dataset. Are there SBERT models on the same domain?

model = "bert-base-nli-mean-tokens"
embedder = SentenceTransformer(model)

c1_embedded = embedder.encode(claim_1)

batch_size = 128
corpus_emb = []


for i in tqdm(range(0, len(corpus_texts), batch_size)):
  batch = corpus_texts[i:i+batch_size]
  emb = embedder.encode(batch)
  corpus_emb.extend(emb)


similarity = cosine_similarity_matrix(c1_embedded, np.array(corpus_emb))

top_index = np.argsort(similarity, axis=0)[::-1][:10]
top_evidences = [(corpus_ids[i], corpus_texts[i]) for i in top_index]

for evidence in top_evidences:
    print(evidence)



100%|██████████| 41/41 [00:37<00:00,  1.10it/s]

('1388704', 'the essence of snps .  single nucleotide polymorphisms (snps) are an abundant form of genome variation, distinguished from rare variations by a requirement for the least abundant allele to have a frequency of 1% or more .  a wide range of genetics disciplines stand to benefit greatly from the study and use of snps .  the recent surge of interest in snps stems from, and continues to depend upon, the merging and coincident maturation of several research areas, i . e .  (i) large-scale genome analysis and related technologies, (ii) bio-informatics and computing, (iii) genetic analysis of simple and complex disease states, and (iv) global human population genetics .  these fields will now be propelled forward, often into uncharted territories, by ongoing discovery efforts that promise to yield hundreds of thousands of human snps in the next few years .  major questions are now being asked, experimentally, theoretically and ethically, about the most effective ways to unlock the

### TODO 2: use Dzeniks/roberta-fact-check and the retrieved evidence to verify the claim. You can use one or multiple evidence.

In [5]:
#!pip install transformers
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [6]:
# TODO: use Dzeniks/roberta-fact-check and the retrieved evidence to verify the claim. You can use one or multiple evidence.

model_name = "Dzeniks/roberta-fact-check"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)


In [8]:
evidence_text = top_evidences[0][1]
print(evidence_text[:70], '...')

encoded_input = tokenizer.encode_plus(
    claim_1,
    evidence_text,
    add_special_tokens=True,
    truncation=True,
    padding="longest",
    return_tensors="pt"
)

input_ids = encoded_input["input_ids"]
attention_mask = encoded_input["attention_mask"]

outputs = model(input_ids=input_ids, attention_mask=attention_mask)
logits = outputs.logits
predicted_labels = torch.argmax(logits, dim=1).flatten().tolist()

prediction = predicted_labels[0]
print("Claim prediction:", bool(prediction))


the essence of snps .  single nucleotide polymorphisms (snps) are an a ...
Claim prediction: True


# Relatio

In [9]:
!pip install relatio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.5/464.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 75.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
# Catch warnings for an easy ride
from relatio import FileLogger
logger = FileLogger(level = 'WARNING')

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator12recordStreamERKNS_7DataPtrENS0_10CUDAStreamE
  warn(f"Failed to load image Python extension: {e}")


In [33]:
#!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv

import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
label_map = {1:"world", 2:"sport", 3:"business", 4:"sci/tech"}
def replace_label(x):
	return label_map[x]
df["label"] = df["label"].apply(replace_label) 
df["doc"] = df["title"] + " " + df["lead"]
df["id"] = df.index
df = df[['id', 'doc']]
df.head()

df = df.sample(n=5000)

### TODO 1: Predict the Semantic Roles in the AG news dataset. 

In [34]:
df.head()

,id,doc
7978,7978,Cisco Ties Microsoft CRM to Your Phone System ...
97007,97007,Date Set for Auction Of Russian Oil Giant The ...
63060,63060,Astros Recover From Oswalt's Poor Start (AP) A...
119773,119773,NBA Wrap: Heat Tame Bobcats to Extend Winning ...
5280,5280,AMD using strained silicon on 90-nanometer chi...


In [ ]:
from relatio import Preprocessor, SRL, extract_roles

# TODO 1: Predict the Semantic Role Labels in the AG news dataset.
# Hint: you can sample a few sentences to accelerate SRL prediction.

preprocessor = Preprocessor(
    spacy_model = "en_core_web_sm",
    remove_punctuation = True,
    remove_digits = True,
    lowercase = True,
    lemmatize = True,
    remove_chars = ["\"",'-',"^",".","?","!",";","(",")",",",":","\'","+","&","|","/","{","}",
                    "~","_","`","[","]",">","<","=","*","%","$","@","#","’"],
    stop_words = [],
    n_process = -1,
    batch_size = 100
)

srl = SRL(
    path = "https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz",
    batch_size = 10,
    cuda_device = -1
)

df = preprocessor.split_into_sentences(
    df, output_path = None, progress_bar = True
)

In [38]:
srl_pred = srl(df['sentence'], progress_bar=True)

roles, _ = extract_roles(
    srl_pred,
    used_roles = ["ARG0","B-V","B-ARGM-NEG","B-ARGM-MOD","ARG1","ARG2"],
    only_triplets = True,
    progress_bar = True
)

for d in roles[0:20]: print(d)

Running SRL...


100%|██████████| 652/652 [24:51<00:00,  2.29s/it]


Extracting semantic roles...


100%|██████████| 6516/6516 [00:01<00:00, 4842.97it/s]

{'ARG0': 'Cisco Systems', 'B-V': 'Ties', 'ARG1': 'Microsoft CRM to Your Phone System'}
{'ARG0': 'Cisco Systems', 'B-V': 'announced', 'ARG1': 'the Cisco CRM Communications Connector'}
{'ARG0': 'the government', 'B-V': 'set', 'ARG1': 'a date to auction off a majority stake in the company # 39;s'}
{'ARG0': 'the government', 'B-V': 'auction', 'ARG1': 'a majority stake in the company # 39;s'}
{'ARG0': "Astros Recover From Oswalt 's Poor Start ( AP ) AP - Roy Oswalt", 'B-V': 'worry', 'ARG1': "the Houston Astros had n't counted on against the St. Louis Cardinals in the NL championship series"}
{'ARG0': 'Astros', 'B-V': 'counted', 'ARG2': 'against the St. Louis Cardinals', 'B-ARGM-NEG': True}
{'ARG0': 'Tame Bobcats', 'B-V': 'Winning', 'ARG1': 'Streak'}
{'ARG0': 'the Miami Heat', 'B-V': 'scorched', 'ARG1': 'the expansion Charlotte Bobcats'}
{'ARG0': 'the Miami Heat', 'B-V': 'extend', 'ARG1': 'their franchise record win streak', 'ARG2': 'to 14 games'}
{'ARG0': '90 - nanometer chips Advanced Micr

### TODO 2: Postprocess the retrieved semantic roles

In [ ]:
# TODO 2: Postprocess the retrieved semantic roles
# Hint: use p.process_roles

process_roles = preprocessor.process_roles(roles)

for d in roles[0:20]: print(d)


Cleaning phrases for role ARG0...
Cleaning phrases for role B-V...


### TODO 3: Extract the named entities that can be recognized from the semantic roles.

In [ ]:
# TODO 3: Extract the named entities that can be recognized from the semantic roles.
# Hint use p.mine_entities


### TODO 4: modeling the narratives

In [ ]:
from relatio.narrative_models import NarrativeModel

# TODO 4: modeling the narratives using NarrativeModel
# Hint: follow the notebook's hyperparameter setting
 

In [ ]:
narative_model.plot_clusters(path = './clusters.pdf')